In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataloader import Loader
import warnings
import os
import time
import math
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings("ignore")

In [2]:
class TinyEEGCNN(nn.Module):
    def __init__(self, in_ch=41):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(in_ch, 64, kernel_size=7, stride=2, padding=3),
            nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),  # -> [B, 128, 1]
        )
        self.fc = nn.Linear(128, 1)  # binary logit

    def forward(self, x):
        h = self.net(x).squeeze(-1)  # [B, 128]
        return self.fc(h).squeeze(-1)  # [B]


d=Loader()
dataloader=d.return_Loader()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TinyEEGCNN().to(device)
print("Model device:", next(model.parameters()).device)

opt = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()


print(device)

Model device: cuda:0
cuda


In [3]:
writer = SummaryWriter("runs/tinycnn") 
epochs = 10
step = 0
for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0.0
        total = 0
        correct = 0

        for batch in dataloader:
            x = batch["x"].to(device, non_blocking=True).float()
            y = batch["y"].to(device, non_blocking=True).float()

            opt.zero_grad(set_to_none=True)
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            opt.step()

            total_loss += loss.item() * x.size(0)
            total += x.size(0)

            with torch.no_grad():
                preds = (torch.sigmoid(logits) >= 0.5).float()
                correct += (preds == y).sum().item()

            writer.add_scalar("train/loss_step", loss.item(), step)
            step += 1

        epoch_loss = total_loss / max(total, 1)
        epoch_acc = correct / max(total, 1)

        writer.add_scalar("train/loss_epoch", epoch_loss, epoch)
        writer.add_scalar("train/acc_epoch", epoch_acc, epoch)
        print(f"Epoch {epoch:02d} | loss={epoch_loss:.4f} | acc={epoch_acc:.3f}")

writer.close()
print("Done. Run: tensorboard --logdir runs")

KeyboardInterrupt: 

cuda available: True
gpu: NVIDIA GeForce RTX 3060
model device: cuda:0


steps per epoch (len(loader)): 4176
avg sec/batch: 0.05610127925872803
estimated epoch time: 234.27894218444825 sec = 3.904649036407471 min
